In [1]:
import pandas as pd
import numpy as np

In [3]:
orders = pd.read_csv("data/olist_orders_dataset.csv")
customers = pd.read_csv("data/olist_customers_dataset.csv")
payments = pd.read_csv("data/olist_order_payments_dataset.csv")
order_items = pd.read_csv("data/olist_order_items_dataset.csv")
reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")


In [5]:
def basic_cleaning(df):
    print("Initial Shape:", df.shape)
    
    # Remove duplicate rows
    df = df.drop_duplicates()
    
    # Remove completely empty rows
    df = df.dropna(how='all')
    
    # Strip column names
    df.columns = df.columns.str.strip()
    
    print("After Cleaning Shape:", df.shape)
    print("-"*50)
    
    return df


In [6]:
orders = basic_cleaning(orders)
customers = basic_cleaning(customers)
payments = basic_cleaning(payments)
order_items = basic_cleaning(order_items)
reviews = basic_cleaning(reviews)


Initial Shape: (99441, 8)
After Cleaning Shape: (99441, 8)
--------------------------------------------------
Initial Shape: (99441, 5)
After Cleaning Shape: (99441, 5)
--------------------------------------------------
Initial Shape: (103886, 5)
After Cleaning Shape: (103886, 5)
--------------------------------------------------
Initial Shape: (112650, 7)
After Cleaning Shape: (112650, 7)
--------------------------------------------------
Initial Shape: (99224, 7)
After Cleaning Shape: (99224, 7)
--------------------------------------------------


In [7]:
# Convert date columns
date_cols = [
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Remove cancelled orders
orders = orders[orders['order_status'] == 'delivered']


In [8]:
customers['customer_city'] = customers['customer_city'].str.lower()
customers['customer_state'] = customers['customer_state'].str.upper()


In [9]:
# Remove negative or zero payments
payments = payments[payments['payment_value'] > 0]

# Check payment types
payments['payment_type'] = payments['payment_type'].str.lower()


In [10]:
# Remove negative prices
order_items = order_items[order_items['price'] > 0]

# Remove negative freight values
order_items = order_items[order_items['freight_value'] >= 0]


In [11]:
# Fill missing review scores with median
reviews['review_score'].fillna(reviews['review_score'].median(), inplace=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_22648\2454413376.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['review_score'].fillna(reviews['review_score'].median(), inplace=True)


In [12]:
# Fill missing review scores with median
reviews['review_score'].fillna(reviews['review_score'].median(), inplace=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_22648\2454413376.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['review_score'].fillna(reviews['review_score'].median(), inplace=True)


In [13]:
Q1 = payments['payment_value'].quantile(0.25)
Q3 = payments['payment_value'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

payments = payments[(payments['payment_value'] >= lower) &
                    (payments['payment_value'] <= upper)]


In [14]:
print("Orders Nulls:\n", orders.isnull().sum())
print("Customers Nulls:\n", customers.isnull().sum())
print("Payments Nulls:\n", payments.isnull().sum())
print("Order Items Nulls:\n", order_items.isnull().sum())
print("Reviews Nulls:\n", reviews.isnull().sum())


Orders Nulls:
 order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
dtype: int64
Customers Nulls:
 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
Payments Nulls:
 order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
Order Items Nulls:
 order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
Reviews Nulls:
 review_id                      0
order_id                       0
review_score                   0
review_comment_title

In [15]:
# Keep only delivered orders
orders = orders[orders['order_status'] == 'delivered']

# Drop rows where customer delivery date is missing
orders = orders.dropna(subset=['order_delivered_customer_date'])


In [16]:
# Merge orders + customers
df = orders.merge(customers, on='customer_id', how='inner')

# Merge payments
df = df.merge(payments, on='order_id', how='inner')

# Merge order items
df = df.merge(order_items, on='order_id', how='inner')

# Merge reviews
df = df.merge(reviews[['order_id', 'review_score']], 
              on='order_id', how='left')


In [17]:
df.to_csv("processed_data/clean_olist_dataset.csv", index=False)
